# [STARTER] Udaplay Project

## Part 01 - Offline RAG

In this part of the project, you'll build your VectorDB using Chroma.

The data is inside folder `project/starter/games`. Each file will become a document in the collection you'll create.
Example.:
```json
{
  "Name": "Gran Turismo",
  "Platform": "PlayStation 1",
  "Genre": "Racing",
  "Publisher": "Sony Computer Entertainment",
  "Description": "A realistic racing simulator featuring a wide array of cars and tracks, setting a new standard for the genre.",
  "YearOfRelease": 1997
}
```


### Setup

In [1]:
# Only needed for Udacity workspace
!pip install chromadb
import importlib.util
import sys

# Check if 'pysqlite3' is available before importing
if importlib.util.find_spec("pysqlite3") is not None:
    import pysqlite3
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

DEPRECATION: Loading egg at d:\anaconda3\lib\site-packages\epajorm-1.0.7-py3.11.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330

[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
import json
import chromadb
from chromadb.utils import embedding_functions
from dotenv import load_dotenv

In [4]:
load_dotenv()

True

### VectorDB Instance

In [6]:
chroma_client = chromadb.PersistentClient(path="chromadb")

### Collection

In [7]:
embedding_fn = embedding_functions.OpenAIEmbeddingFunction(api_key=os.getenv("CHROMA_OPENAI_API_KEY"))

In [8]:
collection = chroma_client.get_or_create_collection(
    name="udaplay",
    embedding_function=embedding_fn
)

### Add documents

In [ ]:
import os
import json
from tqdm import tqdm  # Importa o tqdm para barra de progresso
import chromadb
from chromadb.utils import embedding_functions

# Diretório de dados
data_dir = "games"

# Verifica se diretório existe
if not os.path.exists(data_dir):
    raise FileNotFoundError(f"Diretório '{data_dir}' não encontrado.")

# Inicializa o cliente e coleção do Chroma
embedding_fn = embedding_functions.OpenAIEmbeddingFunction(
    api_key=os.getenv("OPENAI_API_KEY"),  # ou use diretamente a chave como string
    model_name="text-embedding-ada-002"
)
client = chromadb.Client()
collection = client.get_or_create_collection(name="games", embedding_function=embedding_fn)

# Lista apenas arquivos JSON
file_list = [f for f in sorted(os.listdir(data_dir)) if f.endswith(".json")]

# Itera com tqdm para mostrar progresso
for file_name in tqdm(file_list, desc="Processando arquivos JSON"):
    try:
        file_path = os.path.join(data_dir, file_name)
        with open(file_path, "r", encoding="utf-8") as f:
            game = json.load(f)

        content = f"[{game['Platform']}] {game['Name']} ({game['YearOfRelease']}) - {game['Description']}"
        doc_id = os.path.splitext(file_name)[0]

        collection.add(
            ids=[doc_id],
            documents=[content],
            metadatas=[game]
        )
    except Exception as e:
        print(f"❌ Erro ao processar {file_name}: {e}")
